## 5.3 – Object-Oriented Programming 3
### Abstract Classes
Here's a reminder of our vehicle class structure from the previous section:

<table style="border:1px solid black">
<tr style="border-bottom: 1px solid black"><th style="background-color:#FFFFFF">
<p>Vehicle</p>
</th></tr>
<tr><td style="background-color:#F5F5F5">
<p>mileage</p>
</td></tr>
</table>

<img src="./resources/arrow.svg" width=120/>

<table>
<tr><td style="background-color:#FFFFFF;vertical-align:top">

<table style="border:1px solid black">
<tr style="border-bottom: 1px solid black"><th style="background-color:#FFFFFF">
<p><b>Car</b></p>
</th></tr>
<tr><td style="background-color:#F5F5F5">
<p>number_of_doors</p>
</td></tr>
</table>

</td><td style="background-color:#FFFFFF;vertical-align:top">

<table style="border:1px solid black">
<tr style="border-bottom: 1px solid black"><th style="background-color:#FFFFFF">
<p><b>Motorbike</b></p>
</th></tr>
<tr><td style="background-color:#F5F5F5">
<p></p>
</td></tr>
</table>

</td></tr> </table>

Now ask this question: does it ever make sense to create an object of the `Vehicle` class? Sure, there are many vehicles we have not included in our system: boats, aeroplanes; in the right application it might make sense to include these. But then surely we'd create new classes for them too? A `Boat` class and an `Aeroplane` class, both could be subclasses of `Vehicle`.

It's quite natural to fall into this pattern when using inheritance. You have some behaviours you want a group of objects to exhibit, but where the exact implementations may differ (polymorphism), and this naturally leads to a common superclass which itself does not provide those implementations.

We can formalise this in the object-oriented design by calling that class an *abstract class*. An abstract class is one that cannot be instantiated to create objects, but it exists so that subclasses can be created. Abstract classes usually have *abstract methods* as well – methods that are specified only by their *signature*, the parameters and name of the method, but no implementation is provided.

In Python, there are two ways to achieve this. In the older style, you simply make it so that anyone calling the method on the base class will hit an exception. Take a look at the code below:

In [1]:
class Vehicle:
    def __init__(self):
        self.mileage = 0
        
    def sound_effect(self):
        raise NotImplementedError("Abstract method")

    
class Car(Vehicle):
    def __init__(self, number_of_doors):
        super().__init__()
        self.number_of_doors = number_of_doors
        
    def sound_effect(self):
        return "vrooom"
        
        
class Motorbike(Vehicle):
    def sound_effect(self):
        return "brrrrr"
    
    
class Boat(Vehicle):
    def sound_effect(self):
        return "splash"


my_bike = Motorbike()
my_car = Car(5)
my_boat = Boat()

my_vehicles = [my_bike, my_car, my_boat]

for vehicle in my_vehicles:
    print(vehicle.sound_effect())

brrrrr
vrooom
splash


This older style is simpler and reflects the fact that Python wasn't really designed with these ideas in mind, it is a bit of a “hack”. It does not actually stop you from creating an object of the `Vehicle` class, you will just get a runtime error when calling the method.

In [2]:
my_vehicle = Vehicle()
print(my_vehicle.mileage)

0


In addition, there is nothing to stop us creating a subclass which does *not* implement the abstract method, leading to more possible confusion:

In [3]:
class Aeroplane(Vehicle):
    pass

my_plane = Aeroplane()
print(my_plane.sound_effect())

NotImplementedError: Abstract method

Still, you will see this style occasionally so it is good to recognise. Some tools like PyCharm will automatically detect this pattern – if you create a subclass it will show you a hint that you have to implement the abstract method.

#### The ABC of Python
Python also has more formal support in its `abc` module, which stands for Abstract Base Class. It is slightly more complicated to use, and introduces *decoraters* for the first time. These are lines that start with `@` which are placed before a function or class to indicate something special about that function.

The `abc` module can also be used in more than one way, but this is the simple one that I would recommend. From the `abc` module, import the class called `ABC`, and make this the superclass of your abstract class. Also, import the `abstractmethod` decorator, and use it to decorate any abstract methods. The methods may contain some implementation (accessible by subclasses using `super()`) or be blank (using `pass`). 

Importing these items specifically by name rather than using `import abc` avoids having to use the unsightly `abc.ABC` or `@abc.abstractmethod`, but that works too.

Have a look at the code below:

In [4]:
from abc import ABC
from abc import abstractmethod

class Vehicle(ABC):
    def __init__(self):
        self.mileage = 0
        
    @abstractmethod
    def sound_effect(self):
        pass

    
class Car(Vehicle):
    def __init__(self, number_of_doors):
        super().__init__()
        self.number_of_doors = number_of_doors
        
    def sound_effect(self):
        return "vrooom"
        
        
class Motorbike(Vehicle):
    def sound_effect(self):
        return "brrrrr"
    
    
class Boat(Vehicle):
    def sound_effect(self):
        return "splash"


my_bike = Motorbike()
my_car = Car(5)
my_boat = Boat()

my_vehicles = [my_bike, my_car, my_boat]

for vehicle in my_vehicles:
    print(vehicle.sound_effect())

brrrrr
vrooom
splash


So far the results look the same, if slightly more complicated syntax, but notice now we are actually prevented from creating an object from the `Vehicle` class:

In [5]:
my_vehicle = Vehicle()
print(my_vehicle.mileage)

TypeError: Can't instantiate abstract class Vehicle with abstract methods sound_effect

And we are stopped from creating a subclass of an abstract class without implementing the abstract methods, even if we *never* try to call those methods:

In [6]:
class Aeroplane(Vehicle):
    pass

my_plane = Aeroplane()

TypeError: Can't instantiate abstract class Aeroplane with abstract methods sound_effect

### Static and Class Members
Classes are blueprints for objects – understanding this relationship is so fundamental to being able to write object-oriented code. But we need to move on quickly from the beginner concepts to the intermediate ones.

We've mentioned before that in Python *everything* is an object. This is quite literal, even classes are objects! Do you remember writing some unit tests in last week's material? There was a line of code that might have struck you as unusual:
```python
expected_error = [ValueError]
```

Here I have put a `ValueError` into a list, but this is not an object of the type `ValueError`, it is the `ValueError` class *itself*!

In [7]:
expected_error = [ValueError]
print(type(expected_error[0]))

<class 'type'>


Notice that the type of the class object is reported as a `type`. It's possible to create `ValueError` objects:

In [8]:
error = ValueError("Bad arguments")
print(type(error))

<class 'ValueError'>


In practice you normally only deal with error objects when doing error handling:


In [9]:
import math

try:
    math.sqrt(-1)
except ValueError as ve:
    print("Error!")
    print(ve)
    print(type(ve))

Error!
math domain error
<class 'ValueError'>


Don't get too hung up on errors here, the point is more general: if you create a class in Python called `MyClass`, the class name can be used to instantiate objects using the constructor `MyClass()`, but you can also reference the class itself as an object using `MyClass`.

In [10]:
class MyClass:
    pass

print(f"Printing a MyClass object:\t\t {MyClass()}")
print(f"Printing the type of a MyClass object:\t {type(MyClass())}\n")

print(f"Printing the MyClass class:\t\t {MyClass}")
print(f"Printing the type of the MyClass class:\t {type(MyClass)}")

Printing a MyClass object:		 <__main__.MyClass object at 0x7fa548121dc0>
Printing the type of a MyClass object:	 <class '__main__.MyClass'>

Printing the MyClass class:		 <class '__main__.MyClass'>
Printing the type of the MyClass class:	 <class 'type'>


So far, you've seen that objects can have attributes and methods. Within the framework of object-oriented design, we also call these *instance variable* and *instance methods*, to distinguish them from variables and methods which belong *to the class*.

Since everything is an object in Python, it's also possible to give attributes to the class. 

The following code demonstrates a simple class which defines an instance variable:

In [11]:
class MyClassA:
    def __init__(self):
        self.var = 100

# this object has an instance variable    
my_object1 = MyClassA()
print(f"Instance variable: {my_object1.var}\n")

# each instance has its own values for the same variable
my_object1 = MyClassA()
my_object2 = MyClassA()

my_object1.var += 100
print(f"Object 1 instance variable: {my_object1.var}")
print(f"Object 2 instance variable: {my_object2.var}")

Instance variable: 100

Object 1 instance variable: 200
Object 2 instance variable: 100


Whereas the following class contains a *class variable*:

In [12]:
class MyClassB:
    var = 100

# no instance required
print(f"Class variable: {MyClassB.var}\n")

# possible to assign the class to multiple new names, but all point to the same data/variable
rename1 = MyClassB
rename2 = MyClassB

rename1.var += 100

print(f"Rename class variable 1: {rename1.var}")
print(f"Rename class variable 2: {rename2.var}")
print(f"Class variable: {MyClassB.var}\n")

Class variable: 100

Rename class variable 1: 200
Rename class variable 2: 200
Class variable: 200



Interestingly, we can still access the class variable via an instance, though I recommend against this: use the class name instead to be clear.

In [13]:
my_object = MyClassB()
my_object.var

200

I don't recommend making a class which uses the same name for a class variable and an instance variable, but if you do, the class variable is still accessible through the class name:

In [14]:
class MyClassC:
    var = 100
    
    def __init__(self):
        self.var = 200
        
my_object = MyClassC()
print(f"Instance variable: {my_object.var}")
print(f"Class variable: {MyClassC.var}")

Instance variable: 200
Class variable: 100


The point here is that class variables are *shared* by all instances of the class. They *belong to the class* rather than the object.

One classic example of this technique in practice is a class which keeps track of how many instances have been created. This is useful if you want to give each instance a unique ID for example:

In [15]:
class MembershipCard:
    total = 0
    
    def __init__(self, name):
        self.name = name
        self.id = MembershipCard.total
        MembershipCard.total += 1
        

alice_card = MembershipCard("Alice")
bob_card = MembershipCard("Bob")

print(f"Alice's ID is {alice_card.id}")
print(f"Bob's ID is {bob_card.id}")
print(f"There have been a total of {MembershipCard.total} card objects created")

Alice's ID is 0
Bob's ID is 1
There have been a total of 2 card objects created


Here is another example, suppose we are creating a class to represent contracts with temporary contractors that our company takes out. Each contract is good for a certain number of hours, which we need to decrease every time we ask them to do a job. It also contains the name of the contractor. Here's a possible class design with some simple instance variables and an instance method:

In [16]:
class Contract:
    def __init__(self, name, hours):
        self.name = name
        self.hours = hours
        self.active = True
        
    def spend_hours(self, hours_spent):
        if hours_spent > self.hours:
            raise ValueError("Not enough hours on this contact to do that")
            
        self.hours -= hours_spent
        self.active = self.hours > 0
            
    def __str__(self):
        if self.active:
            return f"Contract with {self.name} is still active, {self.hours} hours remaining."
        else:
            return f"Contract with {self.name} is inactive."
            
contract1 = Contract("Sahara", 100)
contract2 = Contract("Irfan", 50)

contract1.spend_hours(40)
contract1.spend_hours(40)
contract1.spend_hours(20)

print(contract1)
print(contract2)

Contract with Sahara is inactive.
Contract with Irfan is still active, 50 hours remaining.


As the demo code shows, each object contains its own values for the instance variables. Changing the values in `contract1` does not change `contract2`.

Now suppose we want to make it so that `spend_hours` returns the amount of money we spend for this contractor to work these hours. For this we obviously need to know the hourly rate, and let's assume in this utopia that everyone is paid equally to do the same work. So while the hourly rate might change from time to time (e.g. the increases you would expect to keep in line with the industry and inflation), it is *not* a piece of information we permit to be different between two separate contracts.

We can add the hourly rate as a class variable, and add a return statement to the `spend_hours` method.

In [17]:
class Contract:
    hourly_rate = 30
    
    def __init__(self, name, hours):
        self.name = name
        self.hours = hours
        self.active = True
        
    def spend_hours(self, hours_spent):
        if hours_spent > self.hours:
            raise ValueError("Not enough hours on this contact to do that")
            
        self.hours -= hours_spent
        self.active = self.hours > 0
        return hours_spent * Contract.hourly_rate
            
    def __str__(self):
        if self.active:
            return f"Contract with {self.name} is still active, {self.hours} hours remaining."
        else:
            return f"Contract with {self.name} is inactive."
            
contract1 = Contract("Sahara", 100)
contract2 = Contract("Irfan", 50)

total_owed = 0
total_owed += contract1.spend_hours(40)
total_owed += contract2.spend_hours(25)

print(f"We owe £{total_owed} for this week's work.")

We owe £1950 for this week's work.


We can also create *class methods* which again belong to the class rather than the object. To do this we need to use the `@classmethod` decorator, and rather than using `self` in the list of parameters, we typically use `cls`. 

Have a look at the `add_inflation` class method below, and the new demo code:

In [18]:
class Contract:
    hourly_rate = 30
    
    def __init__(self, name, hours):
        self.name = name
        self.hours = hours
        self.active = True
        
    def spend_hours(self, hours_spent):
        if hours_spent > self.hours:
            raise ValueError("Not enough hours on this contact to do that")
            
        self.hours -= hours_spent
        self.active = self.hours > 0
        return hours_spent * Contract.hourly_rate
            
    def __str__(self):
        if self.active:
            return f"Contract with {self.name} is still active, {self.hours} hours remaining."
        else:
            return f"Contract with {self.name} is inactive."
        
    @classmethod
    def add_inflation(cls, inflation_percent=1):
        cls.hourly_rate = round(cls.hourly_rate * (1 + inflation_percent/100), 2)
            
contract1 = Contract("Sahara", 100)
contract2 = Contract("Irfan", 50)

Contract.add_inflation(2)
print(f"New hourly rate after inflation: {Contract.hourly_rate}")

total_owed = 0
total_owed += contract1.spend_hours(40)
total_owed += contract2.spend_hours(25)

print(f"We owe £{total_owed} for this week's work.")

New hourly rate after inflation: 30.6
We owe £1989.0 for this week's work.


And finally, Python has one more decorator: `@staticmethod`, which is used for methods that do not use any elements from the class or any instances, but we want to bundle them in this class for the benefit of the structure of the code.

For anyone with a background in Java, or even if you don't, it is worth pointing out that `static` in Java is also used to create class variables, and there is no real distinction between a class method and a static method. Methods in Java do not have the self-referential object name in the list of parameters (`self` and `cls` in Python, `this` in Java).

For this reason, you may occasionally find people talking about *static variables* and *static methods* to refer to the *class* concepts we used here.

There is an example of a `@staticmethod` in the code below. Suppose we find ourselves increasing values by percentages (like inflation) a lot within our `Contract` class, we might want to abstract this logic into its own method, but it does not need to know any information about any contract object or the class itself.

In [19]:
class Contract:
    hourly_rate = 30
    
    def __init__(self, name, hours):
        self.name = name
        self.hours = hours
        self.active = True
        
    def spend_hours(self, hours_spent):
        if hours_spent > self.hours:
            raise ValueError("Not enough hours on this contact to do that")
            
        self.hours -= hours_spent
        self.active = self.hours > 0
        return hours_spent * Contract.hourly_rate
            
    def __str__(self):
        if self.active:
            return f"Contract with {self.name} is still active, {self.hours} hours remaining."
        else:
            return f"Contract with {self.name} is inactive."
        
    @classmethod
    def add_inflation(cls, inflation_percent=1):
        new_rate = Contract.increase_percentage(cls.hourly_rate, inflation_percent)
        cls.hourly_rate = round(new_rate, 2)
        
    @staticmethod
    def increase_percentage(value, percentage):
        return value * (1 + percentage/100)
            
        
contract1 = Contract("Sahara", 100)
contract2 = Contract("Irfan", 50)

Contract.add_inflation(2)
print(f"New hourly rate after inflation: {Contract.hourly_rate}")

total_owed = 0
total_owed += contract1.spend_hours(40)
total_owed += contract2.spend_hours(25)

print(f"We owe £{total_owed} for this week's work.")

New hourly rate after inflation: 30.6
We owe £1989.0 for this week's work.


### Exercise
Continuing the theme from the previous notebooks: go back to your student assignment system design in section 5.1; can you see any places in the class design where you can use abstract classes (with abstract methods), class variables, or class/static methods? Think about each class: does it need instances? Think about each method: does it need information from the object, can it make do with just information from the class, or does it really need neither?

## What Next?
There is so much more to learn about good object-oriented design: there are entire books of design patterns that describe common repeatable ways of setting out your class structures, and there are more advanced features like multiple inheritance, which brings some of its own problems too. But you've seen all that you need to start to make use of object oriented programming. Now you need to practice them, to really ground the concepts we've seen so far. The material in future weeks of the unit will all be using object oriented programming, so we'll introduce new concepts as and when they are useful.

But before that, there's another programming paradigm that has seen a small resurgence in popularity in the last few years: functional programming. Python is not a functional language, but it does support a few nice features that are heavily inspired by functional languages.